In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot = True)


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [11]:
tf.reset_default_graph()

In [18]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

L2_flatten = tf.contrib.layers.flatten(L2)

W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev = 0.01))
b3 = tf.Variable(tf.random_normal([256]))
L3 = tf.nn.relu(tf.matmul(L2_flatten, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev =0.01))
b4 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L3, W4) + b4

cost = tf.losses.softmax_cross_entropy(Y, logits = logits)
train = tf.train.AdamOptimizer(0.001).minimize(cost)


predictions = tf.argmax(tf.nn.softmax(logits), 1)
accuracy = tf.metrics.accuracy(tf.argmax(Y, 1), predictions)

batch_size = 100
with tf.Session() as sess:
    total_batch = int(mnist.train.num_examples/ batch_size)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    for epoch in range(15):
        total_cost = 0
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c, = sess.run([train, cost], feed_dict = {X:batch_xs.reshape(-1, 28, 28, 1), Y: batch_ys, keep_prob: 0.8})
        acc = sess.run([accuracy], feed_dict = {X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels, keep_prob: 1.0})
        print('epoch : {}, cost : {}, acc: {}'.format(epoch, c, acc))
            
            


epoch : 0, cost : 0.053370747715234756, acc: [(0.0, 0.96810001)]
epoch : 1, cost : 0.06267564743757248, acc: [(0.96810001, 0.97430003)]
epoch : 2, cost : 0.0855538621544838, acc: [(0.97430003, 0.97680002)]
epoch : 3, cost : 0.19877442717552185, acc: [(0.97680002, 0.97907501)]
epoch : 4, cost : 0.05799204856157303, acc: [(0.97907501, 0.98093998)]
epoch : 5, cost : 0.11063935607671738, acc: [(0.98093998, 0.98213333)]
epoch : 6, cost : 0.03810318559408188, acc: [(0.98213333, 0.98282856)]
epoch : 7, cost : 0.0006602299981750548, acc: [(0.98282856, 0.98348749)]
epoch : 8, cost : 0.033298783004283905, acc: [(0.98348749, 0.98413336)]
epoch : 9, cost : 0.008241044357419014, acc: [(0.98413336, 0.98462999)]
epoch : 10, cost : 0.02387131005525589, acc: [(0.98462999, 0.98500907)]
epoch : 11, cost : 0.0030551606323570013, acc: [(0.98500907, 0.98540002)]
epoch : 12, cost : 0.06204137206077576, acc: [(0.98540002, 0.98581541)]
epoch : 13, cost : 0.0004125334962736815, acc: [(0.98581541, 0.98612142)]
e

In [20]:
tf.reset_default_graph()

In [21]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

def model_net(x, activation, is_training, reuse = False):
    L1 = tf.layers.conv2d(x, 32, 3, padding='SAME', activation = activation, reuse= reuse, name = 'L1')
    L1 = tf.layers.max_pooling2d(L1, 2, 2)
    L2 = tf.layers.conv2d(L1, 64, 3, padding='SAME', activation = activation, reuse=reuse, name = 'L2')
    L2 = tf.layers.max_pooling2d(L2, 2, 2)
    
    L2_flatten = tf.contrib.layers.flatten(L2)
    
    fc1 = tf.layers.dense(L2_flatten, 256, activation = activation, reuse=reuse, name = 'FC1')
    fc1 = tf.layers.dropout(fc1, 0.2, training = is_training)
    
    fc2 = tf.layers.dense(fc1, 10, reuse=reuse, name = 'output')
    return fc2


In [22]:
logits = model_net(X, tf.nn.relu, True)
test_logits = model_net(X, tf.nn.relu, False, True)
cost = tf.losses.softmax_cross_entropy(Y, logits = logits)
train = tf.train.AdamOptimizer(0.001).minimize(cost)


predictions = tf.argmax(tf.nn.softmax(test_logits), 1)
accuracy = tf.metrics.accuracy(tf.argmax(Y, 1), predictions)

In [24]:
batch_size = 100
with tf.Session() as sess:
    total_batch = int(mnist.train.num_examples/ batch_size)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    for epoch in range(15):
        total_cost = 0
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c, = sess.run([train, cost], feed_dict = {X:batch_xs.reshape(-1, 28, 28, 1), Y: batch_ys})
        acc = sess.run([accuracy], feed_dict = {X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels})
        print('epoch : {}, cost : {}, acc: {}'.format(epoch, c, acc))
            
          

epoch : 0, cost : 0.020873257890343666, acc: [(0.0, 0.98430002)]
epoch : 1, cost : 0.012488684616982937, acc: [(0.98430002, 0.98689997)]
epoch : 2, cost : 0.058932267129421234, acc: [(0.98689997, 0.9878)]
epoch : 3, cost : 0.050051361322402954, acc: [(0.9878, 0.98860002)]
epoch : 4, cost : 0.047780223190784454, acc: [(0.98860002, 0.98923999)]
epoch : 5, cost : 0.03912124037742615, acc: [(0.98923999, 0.98943335)]
epoch : 6, cost : 0.02453668788075447, acc: [(0.98943335, 0.98968571)]
epoch : 7, cost : 0.0036200848408043385, acc: [(0.98968571, 0.98982501)]
epoch : 8, cost : 0.0011123416479676962, acc: [(0.98982501, 0.99014443)]
epoch : 9, cost : 0.004260535817593336, acc: [(0.99014443, 0.98979002)]
epoch : 10, cost : 0.007146528456360102, acc: [(0.98979002, 0.98985457)]
epoch : 11, cost : 0.04281054064631462, acc: [(0.98985457, 0.99010831)]
epoch : 12, cost : 0.0008145206375047565, acc: [(0.99010831, 0.99017692)]
epoch : 13, cost : 0.02385837584733963, acc: [(0.99017692, 0.99017859)]
epoc

In [25]:
tf.reset_default_graph()

In [26]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

def model_net(x, activation, is_training, reuse = False):
    L1 = tf.layers.conv2d(x, 32, 3, padding='SAME', reuse= reuse, name = 'L1')
    L1 = tf.layers.batch_normalization(L1, training=is_training)
    L1 = activation(L1)
    L1 = tf.layers.max_pooling2d(L1, 2, 2)
    
    L2 = tf.layers.conv2d(L1, 64, 3, padding='SAME', activation = activation, reuse=reuse, name = 'L2')
    L2 = tf.layers.batch_normalization(L2, training=is_training)
    L2 = activation(L2)
    L2 = tf.layers.max_pooling2d(L2, 2, 2)
    
    L2_flatten = tf.contrib.layers.flatten(L2)
    
    fc1 = tf.layers.dense(L2_flatten, 256, activation = activation, reuse=reuse, name = 'FC1')
    fc1 = tf.layers.dropout(fc1, 0.2, training = is_training)
    
    fc2 = tf.layers.dense(fc1, 10, reuse=reuse, name = 'output')
    return fc2


In [27]:
logits = model_net(X, tf.nn.relu, True)
test_logits = model_net(X, tf.nn.relu, False, True)
cost = tf.losses.softmax_cross_entropy(Y, logits = logits)
train = tf.train.AdamOptimizer(0.001).minimize(cost)


predictions = tf.argmax(tf.nn.softmax(test_logits), 1)
accuracy = tf.metrics.accuracy(tf.argmax(Y, 1), predictions)

In [ ]:
batch_size = 100
with tf.Session() as sess:
    total_batch = int(mnist.train.num_examples/ batch_size)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    for epoch in range(15):
        total_cost = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c, = sess.run([train, cost], feed_dict = {X:batch_xs.reshape(-1, 28, 28, 1), Y: batch_ys})
        acc = sess.run([accuracy], feed_dict = {X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels})
        print('epoch : {}, cost : {}, acc: {}'.format(epoch, c, acc))          